In [1]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Importing required libraries
import pandas as pd
import time

from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [4]:
pip install --upgrade webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Initializing the driver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service

# Initialize the Firefox driver using WebDriverManager to handle the GeckoDriver
# driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
service = Service(executable_path=GeckoDriverManager().install())
options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(service=service, options=options)
# driver = webdriver.Firefox(executable_path= "/Users/shashnavad/Downloads/Semantic Web Mining Project/geckodriver.exe")
#driver = webdriver.Chrome(r"C:\Users\Amisha\OneDrive\Aegis\NLP\Project\chromedriver.exe")
# Browsing to billboard hot-100 chart on the driver
driver.get("https://www.billboard.com/charts/hot-100/")
dates = [str(i) for i in range(2023,2024)]
#driver.get("https://www.google.com")


In [6]:
import time

artists_data = {} # List to store name of artists
songs_data = {} # List to store top 100 songs

for j in dates:
    link = "https://www.billboard.com/charts/hot-100/{}-07-01/".format(j)
    driver.implicitly_wait(10)
    driver.get(link)

    # Container for referencing the division with top 100 songs
    div = driver.find_elements(By.CLASS_NAME,'o-chart-results-list-row-container')
    songs, artists = [], []
    print(j)
    # Fetching the top 100 songs
    for i in div:
        # Getting the song name
        song = i.find_element(By.CSS_SELECTOR,'li ul li h3').text
        # Getting the artist name
        name = i.find_element(By.CSS_SELECTOR,'li ul li span').text
        # Appending the song and artist name to the lists
        songs.append(song)
        artists.append(name)
    

    artists_data[j] = artists
    songs_data[j] = songs

2023


In [7]:
# Terminating a session and closing the active browser
driver.close()

In [8]:
service = Service(executable_path=GeckoDriverManager().install())
options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(service=service, options=options)

In [9]:
# Browsing to Google
driver.get("https://www.google.com")

In [10]:
# Dataframe to store the song, artist and lyrics
df = pd.DataFrame(columns = ['Top100Year','SongTitle','Artist','LyricsStatus','Lyrics','ReleaseYear','Genre'])

# Fetching song lyrics for top 100 songs
for year in songs_data.keys():
    songs_list = songs_data[year]
    artists_list = artists_data[year]
    # Index Variable
    a = 0
    print(year)
    for song in songs_list:
        # Selenium WebDriverWait is one of the Explicit waits.
        # Explicit waits are confined to a particular web element.
        # Explicit Wait is code you define to wait for a certain condition to occur before proceeding further in the code.
        # It is applied on certain element with defined expected condition and time.
        # This wait is only applied to the specified element. This wait can also throw exception when element is not found.
        # Here we will wait for the google search button element to be clickable.
        # https://www.softwaretestingmaterial.com/webdriverwait-selenium-webdriver/
        element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.NAME, "q")))
        # Entering the search text
        element.send_keys("{} {} lyrics".format(artists_list[a],song))
        # Submiting the form for google search
        element.submit()
        # To prevent captcha being initialized due to bot activity
        time.sleep(10)
        try:
            # Fetching the default lyrics fetched by google using 'lyricfind'
            lyric = driver.find_element(By.CLASS_NAME,'xaAUmb').text
            # If default lyrics is available, set lyricstatus to True
            # Lyric Status True indicates lyrics are available
            status = True
        except:
            # If default lyrics is not available, set lyricstatus to False
            # Lyric Status False indicates lyrics are not available
            status = False
            lyric = None
        try:
            # Fetching genre and release year of the song
            info = driver.find_elements(By.CSS_SELECTOR,'div.rVusze')
            for i in info:
                if i.find_element(By.CLASS_NAME,'w8qArf').text == 'Released:':
                    release_year = i.find_element(By.CLASS_NAME,'LrzXr').text
                if i.find_element(By.CLASS_NAME,'w8qArf').text == 'Genres:':
                    genre = i.find_element(By.CLASS_NAME,'LrzXr').text
            if len(release_year) == 0:
                release_year = None
            if len(genre) == 0:
                genre = None
        except:
            # If genre and year information is not available
            release_year = None
            genre = None
        # Adding the song, artist and lyrics to dataframe
        df.loc[len(df.index)] = [year, song, artists_list[a], status, lyric, release_year, genre]
        a+=1
        # Browsing back to Google search home page
        driver.back()

2023


In [11]:
# Terminating a session and closing the active browser
driver.close()

In [12]:
df

,Top100Year,SongTitle,Artist,LyricsStatus,Lyrics,ReleaseYear,Genre
0,2023,Last Night,Morgan Wallen,True,Last night we let the liquor talk\nI can't rem...,None,None
1,2023,Fast Car,Luke Combs,True,You got a fast car\nAnd I want a ticket to any...,None,None
2,2023,Calm Down,Rema & Selena Gomez,True,"Another banger\nBaby, calm down, calm down\nGi...",None,None
3,2023,Flowers,Miley Cyrus,True,"We were good, we were gold\nKinda dream that c...",None,None
4,2023,All My Life,Lil Durk Featuring J. Cole,True,"Durkio told me he been on some positive shit, ...",None,None
...,...,...,...,...,...,...,...
95,2023,"Angel, Pt. 1","Kodak Black, NLE Choppa, Jimin, JVKE & Muni Long",True,"Angel, don't fly so close to me\nI'll pull you...",2023,"Afrobeats, R&B/Soul, Nigerian R&B, Afropop, UK..."
96,2023,Girl In Mine,Parmalee,True,"In my t-shirt, in my ride\nRunnin' circles in ...",2021,"Afrobeats, R&B/Soul, Nigerian R&B, Afropop, UK..."
97,2023,Moonlight,Kali Uchis,False,None,2023,"Afrobeats, R&B/Soul, Nigerian R&B, Afropop, UK..."
98,2023,Classy 101,Feid x Young Miko,True,To' los día' te imagino\nCómo te debes ver sin...,2023,"Afrobeats, R&B/Soul, Nigerian R&B, Afropop, UK..."


In [13]:
# Converting DataFrame to CSV
df.to_csv('billboard_2023_top_100_song_lyrics.csv', index=False)